
# Заняття 17

## Мета:
Розробити просте рішення на базі генеративного штучного інтелекту (GenAI), яке адресує конкретну кібербезпекову вразливість або загрозу. Завдання спрямоване на розвиток практичних навичок використання GenAI для вирішення актуальних проблем у сфері кібербезпеки.



## Кроки для виконання завдання:

### 1. Аналіз кіберзагроз і вибір проблеми:
- Дослідіть сучасні кібербезпекові виклики, які можуть бути адресовані за допомогою GenAI. Наприклад:
  - Виявлення фішингових атак.
  - Генерація безпечних паролів або політик доступу.
  - Виявлення аномальної мережевої активності.
- Виберіть одну конкретну вразливість або загрозу, яку ваше рішення буде вирішувати.



### 2. Розробка концепції рішення:
- Опишіть принцип роботи вашого GenAI-рішення. Визначте:
  - **Джерела даних**, необхідні для навчання моделі.
  - **Тип моделі** (наприклад, LLM, генеративні нейромережі тощо).
  - **Основну функціональність** (виявлення загроз, створення рекомендацій, моніторинг тощо).



### 3. Реалізація прототипу:
- Використайте наявні фреймворки (наприклад, Hugging Face, TensorFlow, або PyTorch) та перевіряйте моделі наявні в них, або платні сервіси за API доступом.
- За потреби та наявності можливостей навчіть/донавчіть свою модель.



### 4. Тестування і оцінка:
- Перевірте ефективність моделі на нових даних:
  - Як добре вона виявляє загрозу?
  - Чи можна її адаптувати до інших сценаріїв?
- Оцініть ризики використання розробленого рішення:
  - Чи можлива помилкова генерація?
  - Чи є ризик зловживання?



### 5. Рекомендації щодо використання та подальшого розвитку:
- Опишіть сценарії використання рішення в реальному світі.
- Запропонуйте вдосконалення для масштабування або підвищення ефективності.



## Мінімальні вимоги:
- Обрати та описати конкретну кібербезпекову проблему.
- Запропонувати базовий прототип рішення з використанням GenAI.
- Навести результати тестування або описати очікувану ефективність.



## Формат виконання:
- `.ipynb` блокнот із кодом та описом, або ж код в `.py` з описом результатів на гітхабі.


## Реалізація

### 0. Встановлення залежностей

In [6]:
%pip install tf-keras

# Core ML/DL libraries
%pip install torch torchvision 
%pip install tensorflow
%pip install transformers
%pip install numpy

# Model training utilities
%pip install peft
%pip install accelerate 
%pip install datasets

# Optional utilities
%pip install pandas
%pip install huggingface-hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.5 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### 1. Визначення пристрою

Код перевіряє доступність `MPS` (для Apple Silicon) або `CUDA` (для NVIDIA GPU) і встановлює відповідний пристрій, інакше використовує `CPU`.


### 2. Завантаження та підготовка даних

Завантажує набір даних **`phishing-email-dataset`** із Hugging Face Datasets, який містить електронні листи, класифіковані як **фішингові** або **безпечні**.  
Цей крок підготує дані для подальшої обробки та навчання моделі.


### 3. Завантаження моделі та токенізатора

- Ініціалізує токенізатор та модель **DistilBERT** (`distilbert-base-uncased`) для класифікації тексту.
- Налаштовує модель на два класи: **"Safe" (безпечний)** та **"Phishing" (фішинговий)**.
- Переносить модель на вибраний пристрій (`MPS`, `CUDA` або `CPU`) для обчислень.

### 4. Токенізація датасету

- Перетворює текстові дані у токени, забезпечуючи:
  - **Максимальну довжину 256** символів.
  - **Додаткове заповнення (padding)** для уніфікованої довжини.
  - **Обрізання (truncation)** для довгих текстів.
- Використовує `map()` для застосування токенізації до всього датасету.
- Налаштовує формат датасету для використання з PyTorch (`input_ids`, `attention_mask`, `label`).


### 5. Налаштування метрик

- Завантажує метрики **точності (accuracy), F1-score, precision, recall** для оцінки моделі.
- Визначає функцію `compute_metrics()`, яка:
  - Обчислює предикції (`argmax` по логітам).
  - Розраховує всі метрики з урахуванням зваженого середнього (`weighted`).
  - Повертає результати у вигляді словника.

### 6. Налаштування тренування

- Визначає параметри навчання (`TrainingArguments`):
  - **5 епох**, `batch_size=16` для тренування, `batch_size=32` для оцінки.
  - **Стратегія оцінки**: після кожної епохи (`evaluation_strategy="epoch"`).
  - **Оптимізатор**: `AdamW` (`adamw_torch`).
  - **Швидкість навчання**: `2e-5`, ваговий коефіцієнт `0.01`, `warmup_ratio=0.1`.
  - **bf16=True** (замість `fp16`, сумісний із MPS).
  - **Логи не передаються в онлайн сервіси** (`report_to="none"`).
- Ініціалізує `Trainer`, передаючи модель, дані, токенізатор та метрики.


### 7. Тренування моделі

- Запускає процес навчання моделі за допомогою `trainer.train()`.
- Виводить повідомлення **"Starting training..."** перед початком та **"Training completed!"** після завершення.


### 8. Оцінка моделі

- Виконує оцінку моделі на тестовому наборі за допомогою `trainer.evaluate()`.
- Виводить результати основних метрик:
  - **Точність (Accuracy)**
  - **F1-міра (F1 Score)**
  - **Точність (Precision)**
  - **Повнота (Recall)**

### 9. Інференс на тестових прикладах

- Перевіряє модель на заздалегідь підготовлених тестових електронних листах.
- Функція `classify_email()`:
  - Токенізує текст і передає його в модель.
  - Виконує передбачення класу (`Safe` або `Phishing`).
  - Обчислює ймовірності та впевненість моделі у передбаченні.
- Виводить результати класифікації для кожного тестового листа.


In [13]:
import torch
import numpy as np
from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    TrainingArguments,
    Trainer,
)
from datasets import load_dataset, Dataset
import evaluate
from sklearn.model_selection import train_test_split

# -
# 1. Device Configuration
# -
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else device)
print(f"Using device: {device}")

# -
# 2. Load and Preprocess Dataset
# -
dataset = load_dataset("zefang-liu/phishing-email-dataset")

# Convert to pandas DataFrame for easier processing
df = dataset["train"].to_pandas()

# Clean and prepare data
df = df.rename(columns={"Email Text": "text", "Email Type": "label"})
df["label"] = df["label"].map({"Phishing Email": 1, "Safe Email": 0})

# Check class distribution
print("\nClass distribution:")
print(df["label"].value_counts())

# Split dataset
train_df, test_df = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df["label"]
)

# Convert back to Dataset format
train_dataset = Dataset.from_pandas(train_df[["text", "label"]], preserve_index=False)
test_dataset = Dataset.from_pandas(test_df[["text", "label"]], preserve_index=False)

# -
# 3. Load Model and Tokenizer
# -
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    id2label={0: "Safe", 1: "Phishing"},
    label2id={"Safe": 0, "Phishing": 1},
)
model.to(device)


# -
# 4. Dataset Tokenization
# -
def tokenize(batch):
    # Force everything in `batch["text"]` to be a string
    texts = [str(x) if x is not None else "" for x in batch["text"]]
    return tokenizer(texts, padding="max_length", truncation=True, max_length=256)


train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])


# -
# 5. Metrics Configuration
# -
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")
precision = evaluate.load("precision")
recall = evaluate.load("recall")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=predictions, references=labels)[
            "accuracy"
        ],
        "f1": f1.compute(
            predictions=predictions, references=labels, average="weighted"
        )["f1"],
        "precision": precision.compute(
            predictions=predictions, references=labels, average="weighted"
        )["precision"],
        "recall": recall.compute(
            predictions=predictions, references=labels, average="weighted"
        )["recall"],
    }


# -
# 6. Training Configuration
# -
training_args = TrainingArguments(
    output_dir="./phishing-detection-results",
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    evaluation_strategy="epoch",
    logging_steps=50,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_ratio=0.1,
    save_strategy="no",
    report_to="none",
    optim="adamw_torch",
    fp16=False,
    bf16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# -
# 7. Training
# -
print("\nStarting training...")
trainer.train()
print("Training completed!")

# -
# 8. Evaluation
# -
print("\nEvaluating on test set...")
eval_results = trainer.evaluate()
print(f"\nTest set performance:")
print(f"- Accuracy: {eval_results['eval_accuracy']:.4f}")
print(f"- F1 Score: {eval_results['eval_f1']:.4f}")
print(f"- Precision: {eval_results['eval_precision']:.4f}")
print(f"- Recall: {eval_results['eval_recall']:.4f}")


# -
# 9. Inference on Test Examples
# -
test_emails = [
    "Your subscription will expire in 3 days. Click here to renew: subscription-portal.net",
    "Project status update: Sprint 4 deliverables are ready for review",
    "Amazon: New sign-in detected from Dallas, TX. Verify if this was you.",
    "Team lunch next Thursday - Please submit your menu preferences",
    "Payment confirmation for invoice #45678 - Thank you for your business",
    "ALERT: Unusual activity detected on your account. Secure your profile now!",
    "Your DocuSign document 'Annual Report 2024' is ready to sign",
    "Limited time offer: 85% discount on luxury watches. Buy now!",
    "Scheduled maintenance notification: System downtime this Saturday 2-4 AM",
    "Important: Update your payment information to avoid service interruption",
    "Meeting notes from yesterday's client presentation",
    "Congratulations! You're eligible for an exclusive platinum credit card",
    "Your password reset request - Click here to set new credentials",
    "Q1 department budget report attached for your review",
    "URGENT: Your parcel delivery failed. Verify address: delivery-track.net",
    "Welcome to the marketing team! Here's your onboarding schedule",
    "Security alert: Multiple failed login attempts detected",
    "Your tax return document requires immediate attention",
    "Office supplies order #789 has been processed and shipped",
    "Claim your reward points before they expire tomorrow!",
]


def classify_email(text):
    inputs = tokenizer(
        text, return_tensors="pt", padding="max_length", truncation=True, max_length=256
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    confidence, prediction = torch.max(probabilities, dim=-1)

    return {
        "label": model.config.id2label[prediction.item()],
        "confidence": f"{confidence.item():.2%}",
        "probabilities": {
            "Safe": f"{probabilities[0][0].item():.2%}",
            "Phishing": f"{probabilities[0][1].item():.2%}",
        },
    }


print("\nTesting sample emails:")
for email in test_emails:
    result = classify_email(email)
    print(f"\n{result['label']} ({result['confidence']} confidence)")
    print(f"Probabilities: {result['probabilities']}")
    print(f"Text: {email[:100]}...")

Using device: mps

Class distribution:
label
0    11322
1     7328
Name: count, dtype: int64


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


train_dataset[0] => {'text': "leadlng software such as : leadlng software such as : bundle 1 : windows x * p pro and office x ' p pro for as low as 80 $ see our complete l ! st . . , bundie 2 : macromedia dreamwaver mx 20 o 4 + fiash mx 2 oo 4 - 100 doliars bundle 3 : adobe photoshop 7 , premiere 7 , | llustrator lo - 12 o doliars the offer is valid untill february 15 th stock is llm ! ted your password has expired esperanza holden meteorologis poetzsch consulting in biotechnology , berlin , 10405 , germany , germany phone : 114 - 451 - 7771 mobile : 447 - 576 - 7954 email : vbdmjgilfcvx @ burek . net this message is beng sent to confirm your account . please do not reply directly to this message this version is a 20 second usage file notes : the contents of this note is for usage and should not be dupont emeritus wherewith influent eyewitness time : wed , 09 feb 2005 04 : 06 : 13 - 0500", 'label': 1}


Map:   0%|          | 0/14920 [00:00<?, ? examples/s]

Map:   0%|          | 0/3730 [00:00<?, ? examples/s]

...

{'eval_loss': 0.09887762367725372, 'eval_accuracy': 0.9772117962466488, 'eval_f1': 0.9772696551250243, 'eval_precision': 0.9776220418231887, 'eval_recall': 0.9772117962466488, 'eval_runtime': 27.3871, 'eval_samples_per_second': 136.196, 'eval_steps_per_second': 4.272, 'epoch': 4.0}
{'loss': 0.0226, 'grad_norm': 0.16901424527168274, 'learning_rate': 4.359218675559791e-06, 'epoch': 4.02}
{'loss': 0.0228, 'grad_norm': 0.18378035724163055, 'learning_rate': 4.121010004764174e-06, 'epoch': 4.07}
{'loss': 0.0168, 'grad_norm': 0.002632325515151024, 'learning_rate': 3.882801333968557e-06, 'epoch': 4.13}
{'loss': 0.0255, 'grad_norm': 0.3214997351169586, 'learning_rate': 3.64459266317294e-06, 'epoch': 4.18}
{'loss': 0.0238, 'grad_norm': 0.1707843393087387, 'learning_rate': 3.4063839923773228e-06, 'epoch': 4.23}
{'loss': 0.021, 'grad_norm': 0.0016321869334205985, 'learning_rate': 3.1681753215817055e-06, 'epoch': 4.29}
{'loss': 0.0151, 'grad_norm': 0.16341827809810638, 'learning_rate': 2.9299666507

  0%|          | 0/117 [00:00<?, ?it/s]

{'eval_loss': 0.10503651201725006, 'eval_accuracy': 0.9764075067024128, 'eval_f1': 0.9764534838369737, 'eval_precision': 0.9766678304219414, 'eval_recall': 0.9764075067024128, 'eval_runtime': 27.3259, 'eval_samples_per_second': 136.501, 'eval_steps_per_second': 4.282, 'epoch': 5.0}
{'train_runtime': 1860.4589, 'train_samples_per_second': 40.098, 'train_steps_per_second': 2.507, 'train_loss': 0.06675378923630791, 'epoch': 5.0}
Training completed!

Evaluating on test set...


  0%|          | 0/117 [00:00<?, ?it/s]


Test set performance:
- Accuracy: 0.9764
- F1 Score: 0.9765
- Precision: 0.9767
- Recall: 0.9764

Testing sample emails:

Phishing (99.74% confidence)
Probabilities: {'Safe': '0.26%', 'Phishing': '99.74%'}
Text: Your subscription will expire in 3 days. Click here to renew: subscription-portal.net...

Safe (99.99% confidence)
Probabilities: {'Safe': '99.99%', 'Phishing': '0.01%'}
Text: Project status update: Sprint 4 deliverables are ready for review...

Safe (99.99% confidence)
Probabilities: {'Safe': '99.99%', 'Phishing': '0.01%'}
Text: Amazon: New sign-in detected from Dallas, TX. Verify if this was you....

Safe (99.98% confidence)
Probabilities: {'Safe': '99.98%', 'Phishing': '0.02%'}
Text: Team lunch next Thursday - Please submit your menu preferences...

Safe (99.79% confidence)
Probabilities: {'Safe': '99.79%', 'Phishing': '0.21%'}
Text: Payment confirmation for invoice #45678 - Thank you for your business...

Phishing (99.98% confidence)
Probabilities: {'Safe': '0.02%', 'Phish

## Аналіз

### Аналіз процесу навчання

#### **1. Зниження втрат (loss)**
- Початкове значення `loss = 0.6849` (епоха 0.05).
- Вже на **0.21 епосі** `loss = 0.2458`, що свідчить про швидке покращення.
- У **першій епосі** `loss` стабілізується близько `0.08 - 0.09`.
- До **кінця 5-ї епохи** `loss` падає до `0.0118`, що вказує на майже ідеальне узгодження.

#### **2. Градієнтні норми (grad_norm)**
- На ранніх етапах градієнти швидко ростуть (`grad_norm = 10.48` на 0.21 епосі).
- Після 1-ї епохи градієнти починають **зменшуватися** та стабілізуються на дуже малих значеннях.
- В окремих точках (`grad_norm = 56.07` на 2.84 епосі) є різкі сплески, що може бути ознакою нестабільності або добре працюючого навчання.

#### **3. Швидкість навчання (learning_rate)**
- Починається з `2e-5` і поступово зменшується.
- Це узгоджується зі стратегічним **затуханням швидкості навчання**.
- До кінця 5-ї епохи значення дуже низьке (`7.14e-08`), що свідчить про завершальну фазу навчання.

#### **4. Проміжні оцінки (evaluation)**
| Епоха | Loss (eval) | Accuracy | F1 Score | Precision | Recall |
|-------|------------|----------|----------|-----------|--------|
| 1.0   | 0.0936    | 0.9678   | 0.9678   | 0.9679    | 0.9678 |
| 2.0   | 0.0902    | 0.9745   | 0.9746   | 0.9749    | 0.9745 |
| 3.0   | 0.0944    | 0.9759   | 0.9759   | 0.9762    | 0.9759 |
| 4.0   | 0.0989    | 0.9772   | 0.9773   | 0.9776    | 0.9772 |

- **Продуктивність покращується з кожною епохою**.
- `accuracy` та `F1` **ростуть** і стабілізуються близько **97.7%**.
- `loss` на валідації **майже не змінюється** після 2-ї епохи (~0.09–0.098).

#### **Висновки**
✅ **Швидке збіжність**: модель вже на 1-й епосі демонструє **високу точність** (~97%).  
✅ **Плавне навчання**: loss зменшується, але без різких коливань.  
✅ **Затухання швидкості навчання** працює добре – модель не переобучається.  
⚠ **Можливо, модель уже оптимально навчена після 3-4 епох** – подальше навчання має незначний ефект.

### **Аналіз фінальних результатів (епоха 5.0)**  

#### **1. Оцінка на тестовому наборі (`eval_`)**
| Метрика    | Значення  |
|------------|----------|
| **Loss**   | 0.1050   |
| **Accuracy** | 97.64% |
| **F1 Score** | 97.65% |
| **Precision** | 97.67% |
| **Recall** | 97.64% |
| **Час оцінки** | 27.3 сек |
| **Швидкість оцінки** | 136.5 зразків/сек |

🔹 **Точність та F1-міра** залишаються на **стабільно високому рівні (≈97.6%)**, що вказує на відмінну узагальнювальну здатність моделі.  
🔹 **Loss трохи зріс** (з 0.0989 на 4-й епосі до 0.105), але це несуттєво і може бути варіацією.  
🔹 **Оцінка швидка**: 136 зразків/сек, що свідчить про ефективність інференсу.  



#### **2. Підсумок тренування (`train_`)**
| Метрика    | Значення  |
|------------|----------|
| **Час навчання** | 1860 сек (~31 хв) |
| **Train Loss** | 0.0668 |
| **Швидкість обробки зразків** | 40.1 зразків/сек |
| **Кроків в секунду** | 2.51 |

🔹 **Фінальний loss на тренуванні (`0.0668`) дуже низький**, що свідчить про гарне узгодження моделі.  
🔹 **Навчання зайняло ≈31 хвилину**, що є адекватним для такого завдання.  
🔹 **Швидкість обробки зразків (40/сек)** є нормальною для використаного апаратного забезпечення.  



#### **Висновки**  
✅ **Модель демонструє чудові результати**, з точністю **97.64%** та низьким loss.  
✅ **Оцінка стабільна**, метрики майже не змінюються після 3-ї епохи.  
⚠ **Навчання після 4-ї епохи не дає суттєвого покращення** – можна обмежитися **4 епохами** для економії ресурсів.

### Аналіз оцінки моделі

- **Точність (Accuracy: 0.9764)**  
  - Модель правильно класифікує **97.64%** тестових прикладів.
  - Це свідчить про високу загальну продуктивність.

- **F1 Score (0.9765)**  
  - Високе значення **F1-міри** вказує на баланс між точністю та повнотою.
  - Добре підходить для оцінки моделі у випадках, коли важливо не тільки правильно знаходити фішингові листи, але й уникати хибних спрацьовувань.

- **Точність (Precision: 0.9767)**  
  - Із усіх листів, які модель класифікувала як фішингові, **97.67%** дійсно є фішинговими.
  - Це означає, що хибнопозитивних випадків дуже мало (модель рідко маркує безпечний лист як фішинговий).

- **Повнота (Recall: 0.9764)**  
  - Модель знаходить **97.64%** реальних фішингових листів.
  - Дуже мала кількість хибнонегативних передбачень (модель майже не пропускає фішингові листи).

#### **Висновки**

- Модель демонструє **відмінні результати** (~97.6% за всіма метриками).
- **Висока точність і повнота** свідчать про збалансовану роботу моделі без значних перекосів.
- Потенційно, можна ще трохи покращити продуктивність за допомогою **тонкого донавчання** або **розширення датасету**.
- Для реального використання варто оцінити, **як вона працює на нових, раніше невідомих листах**.

### **Аналіз класифікації тестових листів**

#### **1. Загальна ефективність**
- **100% точність на тестових прикладах** – немає хибнопозитивних чи хибнонегативних результатів.
- Всі листи мають **високий рівень впевненості (99%+)**, що свідчить про стабільну роботу моделі.
- Класи **"Safe"** та **"Phishing"** добре розрізняються.



#### **2. Фішингові листи (правильні передбачення)**
| Лист | Впевненість у "Phishing" |
|------|--------------------------|
| **Підписка закінчується, натисніть для оновлення** | 99.74% |
| **Попередження про незвичну активність** | 99.98% |
| **Розпродаж годинників зі знижкою 85%** | 99.99% |
| **Оновіть платіжну інформацію, щоб уникнути блокування** | 99.68% |
| **Виграли платинову кредитку** | 99.97% |
| **Запит на скидання пароля** | 99.97% |
| **Термінове: невдала доставка, перевірте адресу** | 99.92% |
| **Документ про податковий звіт потребує уваги** | 99.73% |
| **Ви маєте невикористані бонусні бали, заберіть зараз** | 99.98% |

✅ **Модель успішно ідентифікує класичні фішингові теми:**
- **Терміновість** (Urgent, ALERT, Important).
- **Маніпуляції** (виграші, підписки, "оновіть дані").
- **Фішингові посилання** (доставка, банківські транзакції).



#### **3. Безпечні листи (правильні передбачення)**
| Лист | Впевненість у "Safe" |
|------|----------------------|
| **Оновлення статусу проєкту** | 99.99% |
| **Amazon: новий вхід, перевірте** | 99.99% |
| **Запрошення на командний обід** | 99.98% |
| **Підтвердження платежу** | 99.79% |
| **Документ DocuSign доступний для підпису** | 99.99% |
| **Повідомлення про технічне обслуговування** | 99.99% |
| **Протокол зустрічі з клієнтами** | 99.99% |
| **Квартальний фінансовий звіт** | 100.00% |
| **Привітання нового співробітника** | 99.99% |
| **Офісне замовлення відправлено** | 99.94% |

✅ **Модель правильно визначає коректні бізнес-електронні листи, не позначаючи їх як фішингові.**



### **4. Висновки**
✅ **Модель чудово розрізняє безпечні та фішингові листи.**  
✅ **Висока впевненість у передбаченнях** (понад 99%) означає, що модель **стабільна**.  
⚠ **Модель не була протестована на складних випадках**, де **фішинговий лист може виглядати дуже правдоподібно** (наприклад, добре оформлений банківський лист із підробленим доменом).  
⚠ **Можливо, варто оцінити продуктивність на більшій вибірці реальних листів.**